In [188]:
# @author: ericherrmann@berkeley.edu

In [189]:
#USER INPUT HERE
modweight = 0.9 #change this value to the weight we use for modified vs avg epv

#change the name of the filename being used as the mod_epv for the current matchup
#don't include the file extension

filename = 'mod_epv_sanjosest' #yes this is clunky but it's late and I'm tired of fixing bugs
from mod_epv_sanjosest import mod_epv as m_epv

teamname = file.split('_')[2] 

print('opponent = ', teamname)

In [170]:
#import statements
import numpy as np
import pandas as pd
#these two imports will always be the same:
from avg_epv import avg_epv as a_epv
from r_conv_pct import conv_pct as r_conv

In [171]:
r_conv = np.matrix(r_conv)

In [172]:
#fn for ceiling division
def ceildiv(a, b):
    return -(-a // b)

#fn for grouping rows into bins of size binsize and taking the average of each bin
def binaverage(matrix, binsize):
    lst = np.array(matrix)
    output = np.zeros((len(lst), (ceildiv(len(lst[0]),binsize))))
    for i in range(0, len(lst)):
        temp = []
        for j in range(0, len(lst[0]), binsize):
            temp2 = lst[i][j:j + binsize]
            temp2 = [x for x in temp2 if x is not None]
            temp.append(sum(temp2) / len(temp2))
        output[i] = np.array(temp)
    return output

In [173]:
#output the average conversion percentage independent of yardline100 bin
print('conversionbydist =\n', binaverage(r_conv, 10))
#not necessary but cool to look at if we want to use this for something

conversionbydist =
 [[0.724]
 [0.614]
 [0.532]
 [0.473]
 [0.432]
 [0.405]
 [0.386]
 [0.371]
 [0.355]]


In [174]:
#place EPVs into 10 yard bins to match conversion percentages
a_epv10 = binaverage(a_epv, 10)
m_epv10 = binaverage(m_epv, 10)

In [175]:
#print 10 yard bin averages
np.set_printoptions(precision=3, suppress = True)
print(a_epv10, '\n')
print(m_epv10)

[[ 4.063  3.261  2.714  2.166  1.619  1.072  0.525 -0.022 -0.569 -0.87 ]
 [ 4.118  3.315  2.768  2.221  1.674  1.127  0.58   0.033 -0.515 -0.843]
 [ 4.181  3.37   2.823  2.276  1.729  1.182  0.634  0.087 -0.46  -0.815]
 [ 4.254  3.425  2.878  2.331  1.783  1.236  0.689  0.142 -0.405 -0.788]
 [ 4.342  3.48   2.932  2.385  1.838  1.291  0.744  0.197 -0.35  -0.761]
 [ 4.454  3.534  2.987  2.44   1.893  1.346  0.799  0.251 -0.296 -0.733]
 [ 4.609  3.589  3.042  2.495  1.948  1.4    0.853  0.306 -0.241 -0.706]
 [ 4.848  3.644  3.097  2.549  2.002  1.455  0.908  0.361 -0.186 -0.679]
 [ 5.3    3.698  3.151  2.604  2.057  1.51   0.963  0.416 -0.132 -0.651]] 

[[ 2.461  1.337  0.638 -0.061 -0.76  -1.459 -2.158 -2.857 -3.556 -3.941]
 [ 2.543  1.407  0.708  0.009 -0.69  -1.389 -2.088 -2.787 -3.486 -3.906]
 [ 2.637  1.477  0.778  0.079 -0.62  -1.319 -2.018 -2.717 -3.416 -3.871]
 [ 2.748  1.547  0.848  0.149 -0.55  -1.249 -1.948 -2.647 -3.346 -3.836]
 [ 2.884  1.617  0.918  0.219 -0.48  -1.179 -1.8

In [176]:
#create matrix compare fn with multiple use cases
def matrixcompare(m1, m2, operation):
    assert np.matrix(m1).shape == np.matrix(m2).shape, 'matrix size mismatch, cannot compare matrices of different sizes'
    lst1 = np.array(m1)
    lst2 = np.array(m2)
    output = np.zeros((len(lst1), len(lst1[0])))
    for i in range(0, len(lst1)):
        for j in range(0, len(lst1[0])):
            if operation == 'normeddiff':
                #calculate the difference between the mod and the average (how much bigger is the mod epv compared to average?)
                output[i][j] = lst2[i][j] - lst1[i][j]
            elif operation == 'combine':
                output[i][j] = (lst1[i][j] * lst2[i][j]) + lst2[i][j]
            else:
                break
        if operation == 'normeddiff':
            #normalize differences by row (normalized to be a percentage between - modweight and modweight)
            romi = -14 #these two values can be adjusted, but essentially they represent the max differences possible in 2 epvs. 
            romx = 14 #the most 2 epvs can be different is 14 or negative 14, which are the floor and ceiling of m_epv-a_epv
            output[i] = ((2 * (1 + modweight))*((np.array(output[i]) - romi) / (romx - romi)) - (1 + modweight))      
    return output

In [177]:
#compare mod epv with avg epv to create a percent of average epv that represents how much different the modded epv is from the norm
epv_diff = matrixcompare(a_epv10, m_epv10, 'normeddiff') #average EPV *FIRST*. 
print(epv_diff)
#multiply this with our average conversion percentages to modify conversion percentage
m_conv = matrixcompare(epv_diff, r_conv, 'combine')

[[-0.217 -0.261 -0.282 -0.302 -0.323 -0.344 -0.364 -0.385 -0.405 -0.417]
 [-0.214 -0.259 -0.28  -0.3   -0.321 -0.341 -0.362 -0.383 -0.403 -0.416]
 [-0.209 -0.257 -0.278 -0.298 -0.319 -0.339 -0.36  -0.381 -0.401 -0.415]
 [-0.204 -0.255 -0.275 -0.296 -0.317 -0.337 -0.358 -0.379 -0.399 -0.414]
 [-0.198 -0.253 -0.273 -0.294 -0.315 -0.335 -0.356 -0.376 -0.397 -0.413]
 [-0.189 -0.251 -0.271 -0.292 -0.313 -0.333 -0.354 -0.374 -0.395 -0.412]
 [-0.176 -0.249 -0.269 -0.29  -0.311 -0.331 -0.352 -0.372 -0.393 -0.411]
 [-0.156 -0.247 -0.267 -0.288 -0.308 -0.329 -0.35  -0.37  -0.391 -0.409]
 [-0.117 -0.245 -0.265 -0.286 -0.306 -0.327 -0.348 -0.368 -0.389 -0.408]]


In [178]:
r_conv = np.array(r_conv)
r_conv100 = pd.Series(np.repeat([j for i in r_conv for j in i ], 10))
m_conv = np.array(m_conv)
m_conv100 = pd.Series(np.repeat([j for i in m_conv for j in i ], 10))

In [179]:
#insert output statement here
np.savetxt("avg_conv_pct.csv", r_conv100, fmt = '%10.5f')
np.savetxt("mod_conv_pct_" + teamname + ".csv", m_conv100, fmt = '%10.5f')

In [180]:
#print values for testing purposes:

print('\n raw:\n')
print(r_conv)
print('\n adjusted:\n')
print(m_conv)


 raw:

[[0.704 0.696 0.711 0.713 0.725 0.73  0.739 0.735 0.752 0.739]
 [0.52  0.608 0.604 0.613 0.619 0.648 0.653 0.65  0.642 0.586]
 [0.415 0.537 0.531 0.536 0.541 0.58  0.579 0.577 0.558 0.469]
 [0.368 0.479 0.483 0.479 0.486 0.524 0.516 0.514 0.496 0.387]
 [0.359 0.435 0.453 0.437 0.447 0.479 0.466 0.464 0.45  0.335]
 [0.367 0.401 0.433 0.408 0.42  0.443 0.428 0.426 0.414 0.312]
 [0.371 0.376 0.416 0.388 0.398 0.413 0.403 0.401 0.384 0.314]
 [0.349 0.358 0.394 0.372 0.375 0.389 0.392 0.39  0.353 0.34 ]
 [0.282 0.344 0.358 0.358 0.346 0.367 0.394 0.394 0.316 0.386]]

 adjusted:

[[0.551 0.515 0.511 0.498 0.491 0.479 0.47  0.452 0.447 0.431]
 [0.409 0.451 0.435 0.429 0.42  0.427 0.417 0.401 0.383 0.343]
 [0.328 0.399 0.383 0.376 0.368 0.383 0.37  0.357 0.334 0.275]
 [0.293 0.357 0.35  0.337 0.332 0.347 0.331 0.32  0.298 0.227]
 [0.288 0.325 0.329 0.309 0.306 0.318 0.3   0.289 0.271 0.197]
 [0.298 0.301 0.316 0.289 0.289 0.295 0.277 0.266 0.251 0.183]
 [0.305 0.283 0.304 0.275 0.274 0